Primero llamamos a las librerías que necesitaremos

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from google.cloud import bigquery


In [9]:
import os

api_json = os.environ.get("GOOGLE_APPLICATION_CREDENTIALS")


In [10]:
client = bigquery.Client(api_json)

In [11]:
# Construct a reference to the "covid19_open_data" dataset
dataset_ref = client.dataset("covid19_open_data", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [12]:
# List all the tables in the "covid19_open_data" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset
for table in tables:  
    print(table.table_id)

compatibility_view
covid19_open_data
